# 사전작업

## 모듈 임포트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import gc

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', 400)

In [4]:
path = './data/'

In [5]:
# load csv
history = pd.read_csv(path + 'historical_transactions.csv')

## Feature Engineering

### util functions

In [6]:
# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [7]:
from scipy import stats
def mode(x):
    return stats.mode(x)[0][0]

In [8]:
def null_cnt(x):
    return np.sum(x.isna() * 1)

In [9]:
def over_550(data):
    return len(data[data > 550])

In [10]:
def ref_date(data):
    ref_month = data.purchase_month - data.month_lag
    
    if ref_month % 12 == 0:
        ref_year = ref_month // 12 - 1
        ref_month = 12
        
    else:
        ref_year = ref_month // 12
        ref_month = ref_month % 12
        
    ref_year = data.purchase_year + ref_year
    return datetime.datetime(ref_year, ref_month, 1, 0, 0, 0)

### train test

In [11]:
# preprocessing train & test
def train_test(debug=False):

    # load csv
    train = pd.read_csv(path + 'train.csv', index_col=['card_id'])
    test = pd.read_csv(path + 'test.csv', index_col=['card_id'])
        
    # null processing
    test.loc['C_ID_c27b4f80f7', 'first_active_month'] = '2017-03'
    
    if debug:
        train = train[:1000]
        test = test[:1000]
    
    # outlier
    train['outliers'] = 0
    train.loc[train['target'] < -30, 'outliers'] = 1

    # set target as nan
    test['target'] = np.nan

    # merge
    df = train.append(test)

    del train, test
    gc.collect()

    # first_active_month
    df['first_active'] = pd.to_datetime(df['first_active_month'])
    df['first_active_year'] = df['first_active'].dt.year
    df['first_active_month'] = df['first_active'].dt.month
    df['first_active_elapsed_time_from_trade'] = (datetime.datetime(2018, 4, 30, 23, 59, 59) - df['first_active']).dt.days
    df['first_active_total_day'] = pd.to_timedelta(df['first_active']).dt.total_seconds() / (60 * 60 * 24)

#     df['first_active_quarter'] = df['first_active'].dt.quarter
#     df['first_active_weekofyear'] = df['first_active'].dt.weekofyear
#     df['first_active_dayofweek'] = df['first_active'].dt.dayofweek
#     df['first_active_elapsed_time_from_today'] = (datetime.datetime.today() - df['first_active']).dt.days

#     df['days_feature1_trade'] = df['feature_1'] * df['first_active_elapsed_time_from_trade']
#     df['days_feature2_trade'] = df['feature_2'] * df['first_active_elapsed_time_from_trade'] 
#     df['days_feature3_trade'] = df['feature_3'] * df['first_active_elapsed_time_from_trade'] 

#     df['days_feature1_trade_ratio'] = df['feature_1'] / df['first_active_elapsed_time_from_trade']
#     df['days_feature2_trade_ratio'] = df['feature_2'] / df['first_active_elapsed_time_from_trade']
#     df['days_feature3_trade_ratio'] = df['feature_3'] / df['first_active_elapsed_time_from_trade']
    
#     for f in ['feature_1','feature_2','feature_3', 'first_active_year', 'first_active_month', 'first_active_total_day']:
#         order_label = df.groupby([f])['outliers'].mean()
#         df[f + '_outlier'] = df[f].map(order_label)

#     df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
#     df['feature_mean'] = df['feature_sum'] / 3
#     df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
#     df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
#     df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)
    
#     df['feature_sum_outlier'] = df['feature_1_outlier'] + df['feature_2_outlier'] + df['feature_3_outlier']
#     df['feature_mean_outlier'] = df['feature_sum_outlier'] / 3
#     df['feature_max_outlier'] = df[['feature_1_outlier', 'feature_2_outlier', 'feature_3_outlier']].max(axis=1)
#     df['feature_min_outlier'] = df[['feature_1_outlier', 'feature_2_outlier', 'feature_3_outlier']].min(axis=1)
#     df['feature_var_outlier'] = df[['feature_1_outlier', 'feature_2_outlier', 'feature_3_outlier']].std(axis=1)

    return df

In [12]:
train_test(debug=True).head(3)

,feature_1,feature_2,feature_3,first_active_month,outliers,target,first_active,first_active_year,first_active_elapsed_time_from_trade,first_active_total_day
card_id,,,,,,,,,,
C_ID_92a2005557,5,2,1,6,0.0,-0.820283,2017-06-01,2017,333,17318.0
C_ID_3d0044924f,4,1,0,1,0.0,0.392913,2017-01-01,2017,484,17167.0
C_ID_d639edf6cd,2,2,0,8,0.0,0.688056,2016-08-01,2016,637,17014.0


In [13]:
train_test().to_csv('./data_feature_engineering/train_test_v5.csv')

### historical_transactions

In [16]:
# preprocessing historical transactions
def historical_transactions(hist_df, debug = False):
    
    if debug:
        hist_df = hist_df[:1000]

    # fillna
    hist_df['category_2'].fillna(1.0,inplace=True)
    hist_df['category_3'].fillna('A',inplace=True)
    hist_df['merchant_id'].fillna(np.nan,inplace=True)
    hist_df['installments'].replace(-1, np.nan,inplace=True)
    hist_df['installments'].replace(999, np.nan,inplace=True)

    # trim
    hist_df['purchase_amount_trim'] = hist_df['purchase_amount'].apply(lambda x: min(x, 0.8))

    # Y/N to 1/0
    hist_df['authorized_flag_label'] = hist_df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    hist_df['category_1_label'] = hist_df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    hist_df['category_3_label'] = hist_df['category_3'].map({'A':0, 'B':1, 'C':2})

    # purchase date
    hist_df['purchase_date'] = pd.to_datetime(hist_df['purchase_date'])
    hist_df['purchase_year'] = hist_df['purchase_date'].dt.year
    hist_df['purchase_month'] = hist_df['purchase_date'].dt.month
    hist_df['purchase_day'] = hist_df['purchase_date'].dt.day
    hist_df['purchase_hour'] = hist_df['purchase_date'].dt.hour
    hist_df['purchase_dayofweek'] = hist_df['purchase_date'].dt.dayofweek
    hist_df['purchase_weekofyear'] = hist_df['purchase_date'].dt.weekofyear
    hist_df['purchase_weekend'] = (hist_df['purchase_date'].dt.weekday >=5).astype(int)
    
    # additional features
    hist_df['price'] = hist_df['purchase_amount'] / hist_df['installments']
    
    hist_df['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    hist_df['Mothers_Day_2017']=(pd.to_datetime('2017-06-04')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    hist_df['fathers_day_2017']=(pd.to_datetime('2017-08-13')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    hist_df['Children_day_2017']=(pd.to_datetime('2017-10-12')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    hist_df['Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    hist_df['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    hist_df['Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-hist_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    
    # new
    hist_df['purchase_date_total_day'] = pd.to_timedelta(hist_df['purchase_date']).dt.total_seconds() / (60 * 60 * 24)
    hist_df['ref_date'] = hist_df.apply(ref_date, axis = 1)
    
    hist_df['month_diff_from_trade'] = ((datetime.datetime(2018, 4, 30, 23, 59, 59) - hist_df['purchase_date']).dt.days)//30
    hist_df['month_diff_from_trade'] += hist_df['month_lag']
    hist_df['month_diff_from_today'] = ((datetime.datetime.today() - hist_df['purchase_date']).dt.days)//30
    hist_df['month_diff_from_today'] += hist_df['month_lag']
    
    # reduce memory usage
    hist_df = reduce_mem_usage(hist_df)
    
    aggs = {}
    aggs['authorized_flag_label'] = [mode, 'sum', 'mean']
    aggs['card_id'] = ['size']
    aggs['city_id'] = [mode, 'nunique']
    aggs['category_1_label'] = [mode, 'sum', 'mean']
    aggs['installments'] = [mode, 'sum', 'mean', 'var', 'max', 'min', null_cnt]
    aggs['category_3_label'] = [mode, 'mean']
    aggs['merchant_category_id'] = [mode, 'nunique']
    aggs['merchant_id'] = ['nunique']
    aggs['month_lag'] = [mode, 'sum', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_date'] = ['max','min']
    aggs['purchase_amount'] = ['sum', 'mean', 'var', 'max', 'min', 'skew', over_550]
    aggs['category_2'] = [mode, 'mean']
    aggs['state_id'] = [mode, 'nunique']
    aggs['subsector_id'] = [mode, 'nunique']
    aggs['purchase_amount_trim'] = ['sum', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_year'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_month'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_day'] = [mode, 'nunique', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_hour'] = [mode, 'nunique', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_dayofweek'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_weekofyear'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_weekend'] = [mode, 'sum', 'mean']
    aggs['price'] = ['sum','mean','max','min','var']
    aggs['Christmas_Day_2017'] = ['mean']
    aggs['Mothers_Day_2017'] = ['mean']
    aggs['fathers_day_2017'] = ['mean']
    aggs['Children_day_2017'] = ['mean']
    aggs['Valentine_Day_2017'] = ['mean']
    aggs['Black_Friday_2017'] = ['mean']
    aggs['Mothers_Day_2018'] = ['mean']
    
    # new
    aggs['purchase_date_total_day'] = ['max','min','mean','var','skew']
    aggs['ref_date'] = [mode]
    aggs['month_diff_from_trade'] = ['max','min','mean','var','skew']
    aggs['month_diff_from_today'] = ['max','min','mean','var','skew']
    
    for col in ['category_2','category_3']:
        hist_df[col+'_mean'] = hist_df.groupby([col])['purchase_amount'].transform('mean')
        hist_df[col+'_min'] = hist_df.groupby([col])['purchase_amount'].transform('min')
        hist_df[col+'_max'] = hist_df.groupby([col])['purchase_amount'].transform('max')
        hist_df[col+'_sum'] = hist_df.groupby([col])['purchase_amount'].transform('sum')
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_min'] = ['mean']
        aggs[col+'_max'] = ['mean']
        aggs[col+'_sum'] = ['mean']
    
    hist_df = hist_df.groupby('card_id').agg(aggs)
    
    # change column name
    hist_df.columns = pd.Index([e[0] + "_" + e[1] for e in hist_df.columns.tolist()])
    hist_df.columns = ['hist_'+ c for c in hist_df.columns]

    hist_df['hist_purchase_date_diff'] = (hist_df['hist_purchase_date_max']-hist_df['hist_purchase_date_min']).dt.days
    hist_df['hist_purchase_date_average'] = hist_df['hist_purchase_date_diff']/hist_df['hist_card_id_size']
    hist_df['hist_purchase_date_uptonow'] = (datetime.datetime.today()-hist_df['hist_purchase_date_max']).dt.days
    hist_df['hist_purchase_date_uptomin'] = (datetime.datetime.today()-hist_df['hist_purchase_date_min']).dt.days

    # reduce memory usage
    hist_df = reduce_mem_usage(hist_df)

    return hist_df

In [17]:
historical_transactions(history, debug=True).head(3)

Memory usage after optimization is: 0.10 MB
Decreased by 64.4%
Memory usage after optimization is: 0.00 MB
Decreased by 54.3%


,hist_authorized_flag_label_mode,hist_authorized_flag_label_sum,hist_authorized_flag_label_mean,hist_card_id_size,hist_city_id_mode,hist_city_id_nunique,hist_category_1_label_mode,hist_category_1_label_sum,hist_category_1_label_mean,hist_installments_mode,hist_installments_sum,hist_installments_mean,hist_installments_var,hist_installments_max,hist_installments_min,hist_installments_null_cnt,hist_category_3_label_mode,hist_category_3_label_mean,hist_merchant_category_id_mode,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_month_lag_mode,hist_month_lag_sum,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_max,hist_month_lag_min,hist_month_lag_skew,hist_purchase_date_max,hist_purchase_date_min,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_skew,hist_purchase_amount_over_550,hist_category_2_mode,hist_category_2_mean,hist_state_id_mode,hist_state_id_nunique,hist_subsector_id_mode,hist_subsector_id_nunique,hist_purchase_amount_trim_sum,hist_purchase_amount_trim_mean,hist_purchase_amount_trim_var,hist_purchase_amount_trim_max,hist_purchase_amount_trim_min,hist_purchase_amount_trim_skew,hist_purchase_year_mode,hist_purchase_year_nunique,hist_purchase_year_mean,hist_purchase_year_max,hist_purchase_year_min,hist_purchase_month_mode,hist_purchase_month_nunique,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_day_mode,hist_purchase_day_nunique,hist_purchase_day_mean,hist_purchase_day_var,hist_purchase_day_max,hist_purchase_day_min,hist_purchase_day_skew,hist_purchase_hour_mode,hist_purchase_hour_nunique,hist_purchase_hour_mean,hist_purchase_hour_var,hist_purchase_hour_max,hist_purchase_hour_min,hist_purchase_hour_skew,hist_purchase_dayofweek_mode,hist_purchase_dayofweek_nunique,hist_purchase_dayofweek_mean,hist_purchase_dayofweek_max,hist_purchase_dayofweek_min,hist_purchase_weekofyear_mode,hist_purchase_weekofyear_nunique,hist_purchase_weekofyear_mean,hist_purchase_weekofyear_max,hist_purchase_weekofyear_min,hist_purchase_weekend_mode,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_purchase_date_total_day_max,hist_purchase_date_total_day_min,hist_purchase_date_total_day_mean,hist_purchase_date_total_day_var,hist_purchase_date_total_day_skew,hist_ref_date_mode,hist_month_diff_from_trade_max,hist_month_diff_from_trade_min,hist_month_diff_from_trade_mean,hist_month_diff_from_trade_var,hist_month_diff_from_trade_skew,hist_month_diff_from_today_max,hist_month_diff_from_today_min,hist_month_diff_from_today_mean,hist_month_diff_from_today_var,hist_month_diff_from_today_skew,hist_category_2_mean_mean,hist_category_2_min_mean,hist_category_2_max_mean,hist_category_2_sum_mean,hist_category_3_mean_mean,hist_category_3_min_mean,hist_category_3_max_mean,hist_category_3_sum_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_0e171c1b48,1,261.0,0.935547,279,277,4,0,0,0.0000,0.0,3.0,0.010750,0.010674,1.0,0.0,0.0,0,0.010750,705,36,72,-8,-1550.0,-5.554688,12.046875,0,-12,0.108337,2018-02-27 13:48:19,2017-02-25 09:38:46,-197.37500,-0.707520,0.004387,-0.296143,-0.745605,4.031250,0.0,4.0,3.695312,13,3,33,17,-197.37500,-0.707520,0.004387,-0.296143,-0.745605,4.031250,2017,2,2017.0,2018,2017,2,12,6.335938,12,1,20,31,15.843750,78.1875,31,1,-0.064514,20,20,15.718750,45.156250,23,0,-1.161133,5,7,3.476562,6,0,8,50,25.578125,52,1,0,112,0.401367,-inf,-inf,-0.566590,-inf,NaN,9.351562,10.835938,15.789062,14.148438,11.390625,13.539062,8.234375,17

In [49]:
historical_transactions(history).to_csv('./data_feature_engineering/history_v1.csv')

Memory usage after optimization is: 2998.48 MB
Decreased by 62.0%
Memory usage after optimization is: 88.79 MB
Decreased by 54.2%


### new_merchant_transactions

In [44]:
# preprocessing new_merchant_transactions
def new_merchant_transactions(debug=False):
    
    # load csv
    new_merchant_df = pd.read_csv(path + 'new_merchant_transactions.csv')
    
    if debug:
        new_merchant_df = new_merchant_df[:1000]

    # fillna
    new_merchant_df['category_2'].fillna(1.0,inplace=True)
    new_merchant_df['category_3'].fillna('A',inplace=True)
    new_merchant_df['merchant_id'].fillna(np.nan,inplace=True)
    new_merchant_df['installments'].replace(-1, np.nan,inplace=True)
    new_merchant_df['installments'].replace(999, np.nan,inplace=True)

    # trim
    new_merchant_df['purchase_amount_trim'] = new_merchant_df['purchase_amount'].apply(lambda x: min(x, 0.8))

    # Y/N to 1/0
    new_merchant_df['authorized_flag'] = new_merchant_df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    new_merchant_df['category_1'] = new_merchant_df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    new_merchant_df['category_3'] = new_merchant_df['category_3'].map({'A':0, 'B':1, 'C':2})

    # purchase date
    new_merchant_df['purchase_date'] = pd.to_datetime(new_merchant_df['purchase_date'])
    new_merchant_df['purchase_year'] = new_merchant_df['purchase_date'].dt.year
    new_merchant_df['purchase_month'] = new_merchant_df['purchase_date'].dt.month
    new_merchant_df['purchase_day'] = new_merchant_df['purchase_date'].dt.day
    new_merchant_df['purchase_hour'] = new_merchant_df['purchase_date'].dt.hour
    new_merchant_df['purchase_dayofweek'] = new_merchant_df['purchase_date'].dt.dayofweek
    new_merchant_df['purchase_weekofyear'] = new_merchant_df['purchase_date'].dt.weekofyear
    new_merchant_df['purchase_weekend'] = (new_merchant_df['purchase_date'].dt.weekday >=5).astype(int)

    # additional features
    new_merchant_df['price'] = new_merchant_df['purchase_amount'] / new_merchant_df['installments']
    
    new_merchant_df['month_diff'] = ((datetime.datetime(2018, 4, 30, 23, 59, 59) - new_merchant_df['purchase_date']).dt.days)//30
    new_merchant_df['month_diff'] += new_merchant_df['month_lag']

    new_merchant_df['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    new_merchant_df['Children_day_2017']=(pd.to_datetime('2017-10-12')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    new_merchant_df['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    new_merchant_df['Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-new_merchant_df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

    # reduce memory usage
    new_merchant_df = reduce_mem_usage(new_merchant_df)
    
    aggs = {}
    aggs['authorized_flag'] = [mode, 'sum', 'mean']
    aggs['card_id'] = ['size']
    aggs['city_id'] = [mode, 'nunique']
    aggs['category_1'] = [mode, 'sum', 'mean']
    aggs['installments'] = [mode, 'sum', 'mean', 'var', 'max', 'min', null_cnt]
    aggs['category_3'] = [mode, 'mean']
    aggs['merchant_category_id'] = [mode, 'nunique']
    aggs['merchant_id'] = ['nunique']
    aggs['month_lag'] = [mode, 'sum', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_date'] = ['max','min']
    aggs['purchase_amount'] = ['sum', 'mean', 'var', 'max', 'min', 'skew', over_550]
    aggs['category_2'] = [mode, 'mean']
    aggs['state_id'] = [mode, 'nunique']
    aggs['subsector_id'] = [mode, 'nunique']
    aggs['purchase_amount_trim'] = ['sum', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_year'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_month'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_day'] = [mode, 'nunique', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_hour'] = [mode, 'nunique', 'mean', 'var', 'max', 'min', 'skew']
    aggs['purchase_dayofweek'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_weekofyear'] = [mode, 'nunique', 'mean', 'max', 'min']
    aggs['purchase_weekend'] = [mode, 'sum', 'mean']
    aggs['price'] = ['sum','mean','max','min','var']
    aggs['month_diff'] = ['max','min','mean','var','skew']
    aggs['Christmas_Day_2017'] = ['mean']
    aggs['Children_day_2017'] = ['mean']
    aggs['Black_Friday_2017'] = ['mean']
    aggs['Mothers_Day_2018'] = ['mean']

    for col in ['category_2','category_3']:
        new_merchant_df[col+'_mean'] = new_merchant_df.groupby([col])['purchase_amount'].transform('mean')
        new_merchant_df[col+'_min'] = new_merchant_df.groupby([col])['purchase_amount'].transform('min')
        new_merchant_df[col+'_max'] = new_merchant_df.groupby([col])['purchase_amount'].transform('max')
        new_merchant_df[col+'_sum'] = new_merchant_df.groupby([col])['purchase_amount'].transform('sum')
        aggs[col+'_mean'] = ['mean']

    new_merchant_df = new_merchant_df.groupby('card_id').agg(aggs)

    # change column name
    new_merchant_df.columns = pd.Index([e[0] + "_" + e[1] for e in new_merchant_df.columns.tolist()])
    new_merchant_df.columns = ['new_'+ c for c in new_merchant_df.columns]

    new_merchant_df['new_purchase_date_diff'] = (new_merchant_df['new_purchase_date_max']-new_merchant_df['new_purchase_date_min']).dt.days
    new_merchant_df['new_purchase_date_average'] = new_merchant_df['new_purchase_date_diff']/new_merchant_df['new_card_id_size']
    new_merchant_df['new_purchase_date_uptonow'] = (datetime.datetime.today()-new_merchant_df['new_purchase_date_max']).dt.days
    new_merchant_df['new_purchase_date_uptomin'] = (datetime.datetime.today()-new_merchant_df['new_purchase_date_min']).dt.days

    # reduce memory usage
    new_merchant_df = reduce_mem_usage(new_merchant_df)

    return new_merchant_df

In [46]:
new_merchant_transactions(debug=True).head(3)

Memory usage after optimization is: 0.06 MB
Decreased by 70.6%
Memory usage after optimization is: 0.02 MB
Decreased by 54.4%


,new_authorized_flag_mode,new_authorized_flag_sum,new_authorized_flag_mean,new_card_id_size,new_city_id_mode,new_city_id_nunique,new_category_1_mode,new_category_1_sum,new_category_1_mean,new_installments_mode,new_installments_sum,new_installments_mean,new_installments_var,new_installments_max,new_installments_min,new_installments_null_cnt,new_category_3_mode,new_category_3_mean,new_merchant_category_id_mode,new_merchant_category_id_nunique,new_merchant_id_nunique,new_month_lag_mode,new_month_lag_sum,new_month_lag_mean,new_month_lag_var,new_month_lag_max,new_month_lag_min,new_month_lag_skew,new_purchase_date_max,new_purchase_date_min,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_skew,new_purchase_amount_over_550,new_category_2_mode,new_category_2_mean,new_state_id_mode,new_state_id_nunique,new_subsector_id_mode,new_subsector_id_nunique,new_purchase_amount_trim_sum,new_purchase_amount_trim_mean,new_purchase_amount_trim_var,new_purchase_amount_trim_max,new_purchase_amount_trim_min,new_purchase_amount_trim_skew,new_purchase_year_mode,new_purchase_year_nunique,new_purchase_year_mean,new_purchase_year_max,new_purchase_year_min,new_purchase_month_mode,new_purchase_month_nunique,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_day_mode,new_purchase_day_nunique,new_purchase_day_mean,new_purchase_day_var,new_purchase_day_max,new_purchase_day_min,new_purchase_day_skew,new_purchase_hour_mode,new_purchase_hour_nunique,new_purchase_hour_mean,new_purchase_hour_var,new_purchase_hour_max,new_purchase_hour_min,new_purchase_hour_skew,new_purchase_dayofweek_mode,new_purchase_dayofweek_nunique,new_purchase_dayofweek_mean,new_purchase_dayofweek_max,new_purchase_dayofweek_min,new_purchase_weekofyear_mode,new_purchase_weekofyear_nunique,new_purchase_weekofyear_mean,new_purchase_weekofyear_max,new_purchase_weekofyear_min,new_purchase_weekend_mode,new_purchase_weekend_sum,new_purchase_weekend_mean,new_price_sum,new_price_mean,new_price_max,new_price_min,new_price_var,new_month_diff_max,new_month_diff_min,new_month_diff_mean,new_month_diff_var,new_month_diff_skew,new_Christmas_Day_2017_mean,new_Children_day_2017_mean,new_Black_Friday_2017_mean,new_Mothers_Day_2018_mean,new_duration_mean,new_duration_min,new_duration_max,new_duration_var,new_duration_skew,new_amount_month_ratio_mean,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew,new_category_2_mean_mean,new_category_3_mean_mean,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,new_purchase_date_uptomin
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_0682c61725,1,2,1,2,19,2,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,222,2,2,1,3,1.500000,0.500000,2,1,NaN,2018-04-09 11:49:24,2018-03-05 09:39:03,-1.293945,-0.646973,0.010162,-0.575684,-0.718262,NaN,0.0,1.0,1.000000,9,1,21,1,-1.293945,-0.646973,0.010162,-0.575684,-0.718262,NaN,2018,1,2018.0,2018,2018,3,2,3.500000,4,3,5,2,7.000000,8.0000,9,5,NaN,9,2,10.000000,2.000000,11,9,NaN,0,1,0.000000,0,0,10,2,12.50000,15,10,0,0,0.000000,-inf,-inf,-inf,-inf,NaN,2,2,2.000000,0.000000,NaN,0.0,0.0,0.000000,50.5000,-1.293945,-1.436523,-1.151367,0.040649,NaN,-0.323486,-0.359131,-0.287842,0.002541,NaN,-0.550293,-0.541504,35,17.500000,303,338
C_ID_086fe1da99,1,22,1,22,69,5,0,1,0.045441,1.0,22.0,1.0,0.0,1.0,1.0,0.0,1,1.0,307,18,22,1,31,1.409180,0.253174,2,1,0.397461,2018-04-29 11:54:06,2018-03-01 14:25:48,-13.101562,-0.595215,0.083862,0.673828,-0.739258,4.375000,0.0,1.0,1.000000,9,3,37,14,-13.101562,-0.595215,0.083862,0.673828,-0.739258,4.375000,2018,1,2018.0,2018,2018,3,2,3.408203,4,3,16,16,17.140625,96.0000,31,1,-0.136963,13,11,13.226562,12.664062,20,6,-0.212280,5,7,3.818359,6,0,11,9,13.00000,17,9,0,11,0.500000,-13.099166,-0.595417,0.673639,-0.739395,0.083862,3,2,2.044922,0.045441,4

In [50]:
new_merchant_transactions().to_csv('./data_feature_engineering/new_v1.csv')

Memory usage after optimization is: 125.43 MB
Decreased by 70.6%
Memory usage after optimization is: 69.42 MB
Decreased by 51.4%


### additional_features

In [9]:
df = pd.read_csv('./data_feature_engineering/train_test_v5.csv')
history_df = pd.read_csv('./data_feature_engineering/history_v1.csv')
new_df = pd.read_csv('./data_feature_engineering/new_v1.csv')

In [10]:
result = pd.merge(df, history_df, on='card_id', how='outer')
result = pd.merge(result, new_df, on='card_id', how='outer')

In [11]:
result.head(3)

,card_id,feature_1,feature_2,feature_3,first_active_month,outliers,target,first_active,first_active_year,first_active_elapsed_time_from_trade,first_active_total_day,hist_authorized_flag_label_mode,hist_authorized_flag_label_sum,hist_authorized_flag_label_mean,hist_card_id_size,hist_city_id_mode,hist_city_id_nunique,hist_category_1_label_mode,hist_category_1_label_sum,hist_category_1_label_mean,hist_installments_mode,hist_installments_sum,hist_installments_mean,hist_installments_var,hist_installments_max,hist_installments_min,hist_installments_null_cnt,hist_category_3_label_mode,hist_category_3_label_mean,hist_merchant_category_id_mode,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_month_lag_mode,hist_month_lag_sum,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_max,hist_month_lag_min,hist_month_lag_skew,hist_purchase_date_max,hist_purchase_date_min,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_skew,hist_purchase_amount_over_550,hist_category_2_mode,hist_category_2_mean,hist_state_id_mode,hist_state_id_nunique,hist_subsector_id_mode,hist_subsector_id_nunique,hist_purchase_amount_trim_sum,hist_purchase_amount_trim_mean,hist_purchase_amount_trim_var,hist_purchase_amount_trim_max,hist_purchase_amount_trim_min,hist_purchase_amount_trim_skew,hist_purchase_year_mode,hist_purchase_year_nunique,hist_purchase_year_mean,hist_purchase_year_max,hist_purchase_year_min,hist_purchase_month_mode,hist_purchase_month_nunique,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_day_mode,hist_purchase_day_nunique,hist_purchase_day_mean,hist_purchase_day_var,hist_purchase_day_max,hist_purchase_day_min,hist_purchase_day_skew,hist_purchase_hour_mode,hist_purchase_hour_nunique,hist_purchase_hour_mean,hist_purchase_hour_var,hist_purchase_hour_max,hist_purchase_hour_min,hist_purchase_hour_skew,hist_purchase_dayofweek_mode,hist_purchase_dayofweek_nunique,hist_purchase_dayofweek_mean,hist_purchase_dayofweek_max,hist_purchase_dayofweek_min,hist_purchase_weekofyear_mode,hist_purchase_weekofyear_nunique,hist_purchase_weekofyear_mean,hist_purchase_weekofyear_max,hist_purchase_weekofyear_min,hist_purchase_weekend_mode,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_month_diff_max,hist_month_diff_min,hist_month_diff_mean,hist_month_diff_var,hist_month_diff_skew,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_duration_mean,hist_duration_min,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_2_min_mean,hist_category_2_max_mean,hist_category_2_sum_mean,hist_category_3_mean_mean,hist_category_3_min_mean,hist_category_3_max_mean,hist_category_3_sum_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,new_authorized_flag_mode,new_authorized_flag_sum,new_authorized_flag_mean,new_card_id_size,new_city_id_mode,new_city_id_nunique,new_category_1_mode,new_category_1_sum,new_category_1_mean,new_installments_mode,new_installments_sum,new_installments_mean,new_installments_var,new_installments_max,new_installments_min,new_installments_null_cnt,new_category_3_mode,new_category_3_mean,new_merchant_category_id_mode,new_merchant_category_id_nunique,new_merchant_id_nunique,new_month_lag_mode,new_month_lag_sum,new_month_lag_mean,new_month_lag_var,new_month_lag_max,new_month_lag_min,new_month_lag_skew,new_purchase_date_max,new_purchase_date_min,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_max,new_purchase_am

In [12]:
result.shape

(325540, 252)

In [13]:
# additional features
def additional_features(df):
    df['hist_first_buy'] = (pd.to_datetime(df['hist_purchase_date_min']) - pd.to_datetime(df['first_active'])).dt.days
    df['hist_last_buy'] = (pd.to_datetime(df['hist_purchase_date_max']) - pd.to_datetime(df['first_active'])).dt.days
    df['new_first_buy'] = (pd.to_datetime(df['new_purchase_date_min']) - pd.to_datetime(df['first_active'])).dt.days
    df['new_last_buy'] = (pd.to_datetime(df['new_purchase_date_max']) - pd.to_datetime(df['first_active'])).dt.days

    date_features=['hist_purchase_date_max','hist_purchase_date_min', 'new_purchase_date_max', 'new_purchase_date_min']

    for f in date_features:
        df[f] = pd.to_datetime(df[f]).astype(np.int64) * 1e-9

    df['card_id_total_size'] = df['new_card_id_size']+df['hist_card_id_size']
    df['card_id_size_ratio'] = df['new_card_id_size']/df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_purchase_amount_sum']+df['hist_purchase_amount_sum']
    df['purchase_amount_mean'] = df['new_purchase_amount_mean']+df['hist_purchase_amount_mean']
    df['purchase_amount_max'] = df['new_purchase_amount_max']+df['hist_purchase_amount_max']
    df['purchase_amount_min'] = df['new_purchase_amount_min']+df['hist_purchase_amount_min']
    df['purchase_amount_ratio'] = df['new_purchase_amount_sum']/df['hist_purchase_amount_sum']
    df['month_diff_mean'] = df['new_month_diff_mean']+df['hist_month_diff_mean']
    df['month_diff_ratio'] = df['new_month_diff_mean']/df['hist_month_diff_mean']
    df['month_lag_mean'] = df['new_month_lag_mean']+df['hist_month_lag_mean']
    df['month_lag_max'] = df['new_month_lag_max']+df['hist_month_lag_max']
    df['month_lag_min'] = df['new_month_lag_min']+df['hist_month_lag_min']
    df['category_1_mean'] = df['new_category_1_mean']+df['hist_category_1_label_mean']
    df['installments_total'] = df['new_installments_sum']+df['hist_installments_sum']
    df['installments_mean'] = df['new_installments_mean']+df['hist_installments_mean']
    df['installments_max'] = df['new_installments_max']+df['hist_installments_max']
    df['installments_ratio'] = df['new_installments_sum']/df['hist_installments_sum']
    df['price_total'] = df['purchase_amount_total'] / df['installments_total']
    df['price_mean'] = df['purchase_amount_mean'] / df['installments_mean']
    df['price_max'] = df['purchase_amount_max'] / df['installments_max']
    df['duration_mean'] = df['new_duration_mean']+df['hist_duration_mean']
    df['duration_min'] = df['new_duration_min']+df['hist_duration_min']
    df['duration_max'] = df['new_duration_max']+df['hist_duration_max']
    df['amount_month_ratio_mean']=df['new_amount_month_ratio_mean']+df['hist_amount_month_ratio_mean']
    df['amount_month_ratio_min']=df['new_amount_month_ratio_min']+df['hist_amount_month_ratio_min']
    df['amount_month_ratio_max']=df['new_amount_month_ratio_max']+df['hist_amount_month_ratio_max']
    df['new_CLV'] = df['new_card_id_size'] * df['new_purchase_amount_sum'] / df['new_month_diff_mean']
    df['hist_CLV'] = df['hist_card_id_size'] * df['hist_purchase_amount_sum'] / df['hist_month_diff_mean']
    df['CLV_ratio'] = df['new_CLV'] / df['hist_CLV']

    return df

In [14]:
additional_features(result).head(3)

,card_id,feature_1,feature_2,feature_3,first_active_month,outliers,target,first_active,first_active_year,first_active_elapsed_time_from_trade,first_active_total_day,hist_authorized_flag_label_mode,hist_authorized_flag_label_sum,hist_authorized_flag_label_mean,hist_card_id_size,hist_city_id_mode,hist_city_id_nunique,hist_category_1_label_mode,hist_category_1_label_sum,hist_category_1_label_mean,hist_installments_mode,hist_installments_sum,hist_installments_mean,hist_installments_var,hist_installments_max,hist_installments_min,hist_installments_null_cnt,hist_category_3_label_mode,hist_category_3_label_mean,hist_merchant_category_id_mode,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_month_lag_mode,hist_month_lag_sum,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_max,hist_month_lag_min,hist_month_lag_skew,hist_purchase_date_max,hist_purchase_date_min,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_skew,hist_purchase_amount_over_550,hist_category_2_mode,hist_category_2_mean,hist_state_id_mode,hist_state_id_nunique,hist_subsector_id_mode,hist_subsector_id_nunique,hist_purchase_amount_trim_sum,hist_purchase_amount_trim_mean,hist_purchase_amount_trim_var,hist_purchase_amount_trim_max,hist_purchase_amount_trim_min,hist_purchase_amount_trim_skew,hist_purchase_year_mode,hist_purchase_year_nunique,hist_purchase_year_mean,hist_purchase_year_max,hist_purchase_year_min,hist_purchase_month_mode,hist_purchase_month_nunique,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_day_mode,hist_purchase_day_nunique,hist_purchase_day_mean,hist_purchase_day_var,hist_purchase_day_max,hist_purchase_day_min,hist_purchase_day_skew,hist_purchase_hour_mode,hist_purchase_hour_nunique,hist_purchase_hour_mean,hist_purchase_hour_var,hist_purchase_hour_max,hist_purchase_hour_min,hist_purchase_hour_skew,hist_purchase_dayofweek_mode,hist_purchase_dayofweek_nunique,hist_purchase_dayofweek_mean,hist_purchase_dayofweek_max,hist_purchase_dayofweek_min,hist_purchase_weekofyear_mode,hist_purchase_weekofyear_nunique,hist_purchase_weekofyear_mean,hist_purchase_weekofyear_max,hist_purchase_weekofyear_min,hist_purchase_weekend_mode,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_month_diff_max,hist_month_diff_min,hist_month_diff_mean,hist_month_diff_var,hist_month_diff_skew,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_duration_mean,hist_duration_min,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_2_min_mean,hist_category_2_max_mean,hist_category_2_sum_mean,hist_category_3_mean_mean,hist_category_3_min_mean,hist_category_3_max_mean,hist_category_3_sum_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,new_authorized_flag_mode,new_authorized_flag_sum,new_authorized_flag_mean,new_card_id_size,new_city_id_mode,new_city_id_nunique,new_category_1_mode,new_category_1_sum,new_category_1_mean,new_installments_mode,new_installments_sum,new_installments_mean,new_installments_var,new_installments_max,new_installments_min,new_installments_null_cnt,new_category_3_mode,new_category_3_mean,new_merchant_category_id_mode,new_merchant_category_id_nunique,new_merchant_id_nunique,new_month_lag_mode,new_month_lag_sum,new_month_lag_mean,new_month_lag_var,new_month_lag_max,new_month_lag_min,new_month_lag_skew,new_purchase_date_max,new_purchase_date_min,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_max,new_purchase_am

In [15]:
additional_features(result).shape

(325540, 285)

In [16]:
result = additional_features(result)

In [17]:
train_df = result[result['target'].notnull()]
test_df = result[result['target'].isnull()]

In [18]:
train_df.set_index('card_id').to_csv('./data/train_v4.csv')
test_df.set_index('card_id').to_csv('./data/test_v4.csv')